In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as pyplot
import os

año = 2022

In [2]:
#GRAFICAS

#FORMATO DE TABLAS
def formato_tabla_energia (tabla):
    tabla.rename(columns={
        'INYECCIONES': 'INYECCIONES FÍSICAS', 
        'RETIROS': 'RETIROS FÍSICOS', 
        'INYECCIONES.1': 'INYECCIONES POR CONTRATO', 
        'RETIROS.1': 'RETIROS POR CONTRATO'
    }, inplace=True)
    
    return tabla

def formato_tabla_potencia (tabla):
    tabla.rename(columns={
        tabla.columns[2]: 'INYECCIONES POR CONTRATO',
        tabla.columns[3]: 'INYECCIONES FÍSICAS',
        tabla.columns[4]: 'RETIROS POR CONTRATO',
        tabla.columns[5]: 'RETIROS FÍSICOS',
        tabla.columns[6]: 'EXCEDENTE DE POTENCIA DE PUNTA',
        tabla.columns[7]: 'DEFICIT DE POTENCIA DE PUNTA'
    }, inplace=True)
    
    return tabla

def formato_tabla_dc (tabla):
    tabla.rename(columns={
        tabla.columns[2]: 'INYECCIONES POR CONTRATO',
        tabla.columns[3]: 'INYECCIONES FÍSICAS',
        tabla.columns[4]: 'RETIROS POR CONTRATO',
        tabla.columns[5]: 'RETIROS FÍSICOS',
        tabla.columns[6]: 'BALANCE ACREEDORES',
        tabla.columns[7]: 'BALANCE DEUDORES'
    }, inplace=True)
    
    return tabla

def formato_tabla (tabla, valorizada):  
    if valorizada:
        for col in tabla.columns:
            if tabla[col].dtype == 'float64':
                tabla[col] = pd.Series(tabla[col]).round(2).fillna(value='')
            elif tabla[col].dtype == 'object':
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
        return tabla
    else:
        for col in tabla.columns:
            if tabla[col].dtype == 'float64':
                if tabla[col].max() > 10000 or tabla[col].min() < -10000:
                    tabla[col] = pd.Series(tabla[col])/1000000
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
            elif tabla[col].dtype == 'object':
                tabla[col] = pd.Series(tabla[col]).fillna(value='')
        return tabla

#EXTRACCIÓN DE DATOS
def leer_tablas (ruta, posicion, valorizada, posicion2=0):
    archivos = os.listdir(ruta[0])
    data = []
    i = 0
    
    for archivo in archivos:
        if valorizada:
            data.append(pd.read_excel(ruta[0]+archivo, sheet_name=ruta[1], header=posicion[1][i] + posicion2, nrows=posicion[1][i], usecols=posicion[2]))
        else:
            data.append(pd.read_excel(ruta[0]+archivo, sheet_name=ruta[1], header=posicion[0], nrows=posicion[1][i], usecols=posicion[2]))
        i += 1
        
    tabla = pd.concat([d for d in data])
    tabla = formato_tabla(tabla, valorizada)
    
    return tabla

In [3]:
#ENERGÍA

#DATOS DE ENERGÍA
ruta_energia = ['/mnt/c/Users/aaria/Documents/informe/TEE/', 'BALANCE TOTAL']
archivos_energia = os.listdir(ruta_energia[0])
dimensiones_energia = []

for archivo in archivos_energia:
    dimensiones_energia.append(pd.read_excel(ruta_energia[0]+archivo, sheet_name=ruta_energia[1], usecols='XFD', skiprows=0, nrows=1, header=None, names=['value']).iloc[0]['value'])

posiciones_energia = [12, dimensiones_energia, 'D:L']
#posiciones_energia[1][0]
#BALANCE FÍSICO
energia_fisica = leer_tablas(ruta_energia, posiciones_energia, False)
formato_tabla_energia(energia_fisica)
energia_fisica = energia_fisica.style.format(precision=3).hide(axis="index")

#BALANCE VALORIZADO
energia_valorizada = leer_tablas(ruta_energia, posiciones_energia, True, 22)
formato_tabla_energia(energia_valorizada)
energia_valorizada = energia_valorizada.style.format(thousands=',', precision=2).hide(axis='index')

In [6]:
#POTENCIA

#DATOS DE POTENCIA
ruta_potencia = ['/mnt/c/Users/aaria/Documents/informe/TEP/', 'BALANCE']
archivos_potencia = os.listdir(ruta_potencia[0])
dimensiones_potencia = []

for archivo in archivos_potencia:
    dimensiones_potencia.append(pd.read_excel(ruta_potencia[0]+archivo, sheet_name=ruta_potencia[1], usecols='GC', skiprows=0, nrows=1, header=None, names=['value']).iloc[0]['value'])

posiciones_potencia = [15, dimensiones_potencia, 'C:J']

#BALANCE FÍSICO
potencia_fisica = leer_tablas(ruta_potencia, posiciones_potencia, False)
formato_tabla_potencia(potencia_fisica)
potencia_fisica = potencia_fisica.style.format(precision=3).hide(axis="index")

#BALANCE VALORIZADO
potencia_valorizada = leer_tablas(ruta_potencia, posiciones_potencia, True, 27)
formato_tabla_potencia(potencia_valorizada)
potencia_valorizada = potencia_valorizada.style.format(thousands=',', precision=2).hide(axis='index')

In [7]:
#DERECHO DE CONEXIÓN
resolucion_tdc = 'SIE-118-2021-PJ'

#DATOS DE DERECHO DE CONEXIÓN
ruta_tdc = ['/mnt/c/Users/aaria/Documents/informe/TDC/', 'BALANCE']
archivos_tdc = os.listdir(ruta_tdc[0])
dimensiones_tdc = []

for archivo in archivos_tdc:
    dimensiones_tdc.append(pd.read_excel(ruta_tdc[0]+archivo, sheet_name=ruta_tdc[1], usecols='AR', skiprows=0, nrows=1, header=None, names=['value']).iloc[0]['value'])

posiciones_tdc = [15, dimensiones_tdc, 'C:J']

#BALANCE DC
tdc = leer_tablas(ruta_tdc, posiciones_tdc, True, 20)
formato_tabla_dc(tdc)
tdc = tdc.style.format(thousands=",", precision=2).hide(axis='index')

In [8]:
#TABLAS DE REGULACIÓN DE FRECUENCIA
resolucion_crf = 'SIE-120-2021-MEM'

In [9]:
#TABLAS DE DESPACHO FORZADO
resolucion_cdf = 'SIE-119-2021-MEM'

In [10]:
#TABLAS DE DESVÍO DE LA OPERACIÓN
resoluciones_cdo = ['SIE 374-2012', 'SIE-018-2013-MEM', 'SIE-041-2013-MEM']

# INTRODUCCIÓN
</br>
<div style="text-align: justify">
El presente informe contiene un resumen estadístico de las inyecciones y retiros de los generadores, los retiros de los distribuidores y de los Usuarios No Regulados, así como las correspondientes Transacciones Económicas en el Mercado Eléctrico Mayorista, destacando los pagos al propietario del sistema de transmisión, los pagos realizados por concepto de servicios de regulación de frecuencia y los pagos resultantes de la aplicación del mecanismo de compensación de unidades generadoras por despacho forzado y compensación por desvío, conforme a lo establecido en la Resolución SIE-119-2021-MEM y en la Resolución SIE-041-2013-MEM, respectivamente.
</div>

## 1. RESUMEN EJECUTIVO TRANSACCIONES ECONÓMICAS
</br>
<div style="text-align: justify">
En el año 2022:
<ul>
    <li>El promedio de los Costos Marginales de Energía Activa de Corto Plazo para el sistema principal alcanzó el valor {{cmg_promedio}}, representando {{cambio_cmg_año_anterior}} de {{cambio_cmg_año_anterior_porcentaje}} con relación al promedio del año anterior para el mismo mes ({{cmg_año_anterior}}) y {{cambio_cmg_mes_anterior}} de {{cambio_cmg_mes_anterior_porcentaje}} con relación al mes anterior ({{cmg_mes_anterior}}). El Costo Marginal de Potencia de Punta fue {{cmg_potencia}}, y el Derecho de Conexión Unitario fue de {{dc_unitario}}.</li>
</div>

## 2. OBJETIVO

## 3. TRANSACCIONES ECONÓMICAS DE ENERGÍA
### 3.1 Balance de Energía de los Agentes del MEM

## 4. TRANSACCIONES PROVISIONALES DE POTENCIA

## 5. PEAJE DE TRANSMISIÓN

## 6. SERVICIO DE REGULACIÓN DE FRECUENCIA

## 7. COMPENSACIÓN POR DESPACHO FORZADO SEGÚN RESOLUCIÓN {{resolucion_cdf}}

## 8. COMPENSACIÓN POR DESVÍO SEGÚN RESOLUCIONES {{resoluciones_cdo[0]}}, {{resoluciones_cdo[1]}} Y {{resoluciones_cdo[2]}}

## 9. RESUMEN DE LAS TRANSACCIONES DEL MEM EN EL {{año}}

## 10. COSTOS MARGINALES DE CORTO PLAZO DE ENERGÍA

## 11. INDICADORES DEL MEM

## 12. ASPECTOS RELEVANTES